## **Images & masks directory list**

In [ ]:
# make lists for image and mask directories
import glob

images_dir = '/outputs/dataset/images'
masks_dir = '/outputs/dataset/masks'

image_paths = sorted(glob.glob(f'{images_dir}/*'))
mask_paths = sorted(glob.glob(f'{masks_dir}/*'))

print(f'total images: {len(image_paths)}')
print(f'total masks: {len(mask_paths)}')
print(image_paths[:5])
print(mask_paths[:5])

## **Prepare Data**

In [2]:
import random

# number of validation samples
val_samples = 50

random.Random(1337).shuffle(image_paths)
random.Random(1337).shuffle(mask_paths)

# Split our img paths into a training and a validation set
train_image_paths = image_paths[:-val_samples]
train_mask_paths = mask_paths[:-val_samples]

val_image_paths = image_paths[-val_samples:]
val_mask_paths = mask_paths[-val_samples:]

In [3]:
from dataset import BuildingDataset

img_size = (512, 512)
batch_size = 16

# Instantiate data Sequences for each split
train_gen = BuildingDataset(batch_size,
                            img_size,
                            train_image_paths,
                            train_mask_paths)

val_gen = BuildingDataset(batch_size,
                          img_size,
                          val_image_paths,
                          val_mask_paths)

## **Create and compile the model**

In [4]:
activation_function = 'sigmoid'
num_classes = 1

from models import UNet
model = UNet(num_classes = num_classes,
         image_size = img_size[0], 
         img_channels = 3,
         activation_fn = 'sigmoid')

# from models import DeepUNet
# model =  DeepUNet(num_classes=num_classes,
#              image_size=img_size[0],
#              img_channels=3,
#              activation_fn = activation_function,
#              n_filters_start = 32)

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from loss import bce_dice_loss, dice_coef

loss = bce_dice_loss # can use 'binary_crossentropy'
optimizer = Adam()

# compile model
model.compile(optimizer = optimizer, 
              loss = loss,
              metrics = [Precision(), 
                        Recall(), 
                        dice_coef])

## **Train the model**

In [ ]:
# Train the model, doing validation at the end of each epoch.
epochs = 50

model.fit(train_gen, 
          epochs=epochs, 
          validation_data=val_gen,
          verbose = 1)

## **Save model file**

In [ ]:
import os

# create folder to save weights
weights_save_folder = 'outputs/weights'
os.makedirs(weights_save_folder, exist_ok = True)

model.save(f'{weights_save_folder}/my_model.h5')

## **Inference on a single tile**

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

threshold = 0.95

test_img_dir = 'test_image.png'
test_mask_dir = 'test_mask.png'

# read test image and mask
test_image = Image.open(test_img_dir)
test_mask = Image.open(test_mask_dir)

# make predictions
pred_img = np.expand_dims(test_image, axis = 0)
test_preds = model.predict(pred_img)
test_preds = np.squeeze(test_preds)
test_preds = test_preds > threshold

# plot the results
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)

ax1.imshow(test_image)
ax1.title.set_text('Image')

ax2.imshow(test_preds)
ax2.title.set_text('Predictions')

ax3.imshow(test_mask)
ax3.title.set_text('Ground Truth')